In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [2]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


E0000 00:00:1734940197.565864   17501 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734940197.578506   17501 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is configured properly.
Num GPUs Available: 1


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [4]:
f.predictions('nvda')

I0000 00:00:1734940210.769995   17501 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1734940210.770261   17501 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14223 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1734940214.825534   17679 cuda_dnn.cc:529] Loaded cuDNN version 90300


NVDA 5m Interval Timestamp: 2024-12-23 02:49:59


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,down,up
kelly_1:2.5,0.141147,0.132438,0.136548,0.102994,0.114667
prob_up,0.109735,0.07791,0.218853,0.125,1.0
prob_static,0.370716,0.413108,0.339967,0.125,0.0
prob_down,0.519549,0.508982,0.44118,0.75,0.0
precision,0.386534,0.380313,0.383249,0.359281,0.367619
recall,0.32563,0.357143,0.317227,0.252101,0.810924
f1,0.353478,0.368364,0.347126,0.296296,0.505898
support,"[476.0, 474.0, 476.0]","[476.0, 474.0, 476.0]","[476.0, 474.0, 476.0]","[476.0, 474.0, 476.0]","[476.0, 474.0, 476.0]"


NVDA 15m Interval Timestamp: 2024-12-23 02:50:52


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,down,static,up,static
kelly_1:2.5,0.272626,0.086957,0.241341,0.131646,0.222222
prob_up,0.245587,0.149268,0.238344,0.5,0.0
prob_static,0.453365,0.272447,0.402684,0.25,1.0
prob_down,0.301048,0.578285,0.358972,0.25,0.0
precision,0.480447,0.347826,0.458101,0.379747,0.444444
recall,0.547771,0.352201,0.522293,0.375,0.407643
f1,0.511905,0.35,0.488095,0.377358,0.425249
support,"[160.0, 157.0, 159.0]","[160.0, 157.0, 159.0]","[160.0, 157.0, 159.0]","[160.0, 157.0, 159.0]","[160.0, 157.0, 159.0]"


...

,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,static,down
kelly_1:2.5,0.140406,0.095833,0.114088,0.064249,0.066308
prob_up,0.286035,0.257921,0.346231,0.25,0.0
prob_static,0.316689,0.287506,0.225901,0.375,0.0
prob_down,0.397277,0.454574,0.427868,0.375,1.0
precision,0.386005,0.354167,0.367206,0.331606,0.333077
recall,0.394919,0.353349,0.367206,0.442396,1.0
f1,0.390411,0.353757,0.367206,0.379072,0.499711
support,"[433.0, 434.0, 433.0]","[433.0, 434.0, 433.0]","[433.0, 434.0, 433.0]","[433.0, 434.0, 433.0]","[433.0, 434.0, 433.0]"


NVDA 1wk Interval Timestamp: 2024-12-23 02:53:01


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,up,down
kelly_1:2.5,0.066667,0.106024,0.047222,0.034483,0.063158
prob_up,0.130905,0.047101,0.309092,0.5,0.0
prob_static,0.084504,0.036568,0.254762,0.25,0.0
prob_down,0.784591,0.916331,0.436146,0.25,1.0
precision,0.333333,0.361446,0.319444,0.310345,0.330827
recall,0.306818,0.340909,0.261364,0.303371,1.0
f1,0.319527,0.350877,0.2875,0.306818,0.497175
support,"[89.0, 89.0, 88.0]","[89.0, 89.0, 88.0]","[89.0, 89.0, 88.0]","[89.0, 89.0, 88.0]","[89.0, 89.0, 88.0]"


NVDA 1mo Interval Timestamp: 2024-12-23 02:53:20


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,up,up,static,static
kelly_1:2.5,0.144444,0.16,0.17931,0.05,0.082759
prob_up,0.229161,0.551106,0.426485,0.25,0.0
prob_static,0.570684,0.309735,0.330181,0.5,1.0
prob_down,0.200155,0.13916,0.243335,0.25,0.0
precision,0.388889,0.4,0.413793,0.321429,0.344828
recall,0.35,0.631579,0.631579,0.45,1.0
f1,0.368421,0.489796,0.5,0.375,0.512821
support,"[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')